In [21]:
import pandas as pd 
import numpy as np

# Dataframe de la base de données sur les incendies: 
df_incendies = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\Incendies.csv", sep = ';')

# Dataframe avec la base de données permettant de faire la correspondance commune-coordonnées GPS :
df_communes = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\communes.csv")
df_communes = df_communes[['code_commune_INSEE','latitude','longitude','nom_region']]
df_communes.columns = ['code_commune_INSEE', 'lat', 'lon', 'region']

# Dataframe avec la base de données meteonet pour le quart sud-est :
df_meteo_SE = pd.read_csv(r'C:\Users\felix\Desktop\ensae-prog2A\data\meteonet\ground_stations\SE2018.csv')

# Dataframe avec juste les stations et leurs coordonnées GPS :
df_stations_SE = df_meteo_SE[['number_sta','lat','lon']]


In [23]:
df_stations_SE = df_stations_SE.drop_duplicates()
liste_regions = ['Corse', "Provence-Alpes-Côte d'Azur", "Occitanie", "Auvergne-Rhône-Alpes", 'Nouvelle-Aquitaine']
df_communes = df_communes[df_communes['region'].isin(liste_regions)]

def distance(lat1,lon1,lat2,lon2):
    """ INPUT : les coordonnées GPS de deux points
        OUTPUT : la distance entre ces deux points à vol d'oiseau
    """
    return 6371*np.arccos(np.sin(lat1*np.pi/180)*np.sin(lat2*np.pi/180)+np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.cos((lon2-lon1)*np.pi/180))

# créer une colonne vide "commune" :
df_stations_SE['commune']=""
# créer la colonne "distance station-commune" exprimée en km, arbitrairement égale à 0 pour l'instant :
df_stations_SE['distance station-commune (km)']=0
display(df_stations_SE.head())

,number_sta,lat,lon,commune,distance station-commune (km)
0,1027003,45.83,5.11,,0
1,1033002,46.09,5.81,,0
2,1034004,45.77,5.69,,0
3,1072001,46.20,5.29,,0
4,1089001,45.98,5.33,,0


In [26]:
#associer à chaque station la commune la plus proche
for i in df_stations_SE.index:
    lat1 = df_stations_SE['lat'][i]
    lon1 = df_stations_SE['lon'][i]
    s= 100 #initialise une distance arbitraire 
    plusprochecommune=0
    for j in df_communes.index:
        lat2 = df_communes['lat'][j]
        lon2 = df_communes['lon'][j]
        d = distance(lat1,lon1,lat2,lon2)
        if d <= s:
            s = d
            plusprochecommune = df_communes['code_commune_INSEE'][j]
    df_stations_SE['commune'][i] = plusprochecommune
    df_stations_SE['distance station-commune (km)'][i] = s

      

C:\Users\felix\AppData\Local\Temp\ipykernel_19204\516345226.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_SE['commune'][i] = plusprochecommune
C:\Users\felix\AppData\Local\Temp\ipykernel_19204\516345226.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_SE['commune'][i] = plusprochecommune
C:\Users\felix\AppData\Local\Temp\ipykernel_19204\516345226.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

In [31]:
df_stations_SE = df_stations_SE.rename(columns = {'commune' :'code_commune_INSEE' })
df_incendies = df_incendies.rename(columns = {'Code INSEE' : 'code_commune_INSEE' })

,number_sta,lat,lon,code_commune_INSEE,distance station-commune (km)
0,1027003,45.83,5.11,1027,0.211264
1,1033002,46.09,5.81,1033,2.523936
2,1034004,45.77,5.69,1227,2.060344
3,1072001,46.20,5.29,1369,0.872700
4,1089001,45.98,5.33,1089,1.900183


,Année,Numéro,Département,code_commune_INSEE,Nom de la commune,Date de première alerte,Origine de l'alerte,Moyens de première intervention,Surface parcourue (m2),Surface forêt (m2),...,Décès ou bâtiments touchés,Nombre de décès,Nombre de bâtiments totalement détruits,Nombre de bâtiments partiellement détruits,Hygrométrie (%),Vitesse moyenne du vent (Km/h),Direction du vent,Température (°C),Précision de la donnée,Présence d'un contour valide
0,2016,1,12,12153,Montjaux,2016-01-24 14:55:00,Indéterminé,NaN,6000,0.0,...,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non
1,2016,2,12,12198,Rieupeyroux,2016-01-27 17:56:00,Indéterminé,NaN,20,0.0,...,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non
2,2016,4,12,12163,Murasson,2016-02-21 18:55:00,Indéterminé,NaN,200000,0.0,...,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non
3,2016,7,43,43170,Saint-Beauzire,2016-03-14 18:23:00,Indéterminé,NaN,100,0.0,...,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non
4,2016,8,12,12219,Sainte-Eulalie-d'Olt,2016-03-14 15:10:00,Indéterminé,NaN,1500,0.0,...,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non


In [35]:
# ajout du code commune INSEE dans la base de la météo :
df_meteo_SE_new = pd.merge(df_meteo_SE, df_stations_SE[['number_sta','code_commune_INSEE']], on = 'number_sta', how = 'left')
df_meteo_SE_new.head(30)

,number_sta,lat,lon,height_sta,date,dd,ff,precip,hu,td,t,psl,code_commune_INSEE
0,1027003,45.830,5.110,196.0,20180101 00:00,NaN,NaN,NaN,90.0,279.15,280.65,NaN,1027
1,1027003,45.830,5.110,196.0,20180101 00:00,NaN,NaN,NaN,90.0,279.15,280.65,NaN,1027
2,1033002,46.090,5.810,350.0,20180101 00:00,30.0,2.7,0.2,89.0,277.65,279.35,NaN,1033
3,1033002,46.090,5.810,350.0,20180101 00:00,30.0,2.7,0.2,89.0,277.65,279.35,NaN,74235
4,1034004,45.770,5.690,330.0,20180101 00:00,190.0,1.4,0.0,86.0,278.25,280.45,NaN,1227
5,1034004,45.770,5.690,330.0,20180101 00:00,190.0,1.4,0.0,86.0,278.25,280.45,NaN,1034
6,1072001,46.200,5.290,260.0,20180101 00:00,NaN,NaN,0.0,NaN,NaN,280.25,NaN,1369
7,1072001,46.200,5.290,260.0,20180101 00:00,NaN,NaN,0.0,NaN,NaN,280.25,NaN,1369
8,1089001,45.980,5.330,252.0,20180101 00:00,130.0,1.3,0.0,94.0,279.75,280.65,101760.0,1089
9,1089001,45.980,5.330,252.0,20180101 00:00,130.0,1.3,0.0,94.0,279.75,280.65,101760.0,1089
